In [1]:
%env HDFS_PATH /user/dominospark/small_data
%env HADOOP_HOME /usr/lib/hadoop
%env HADOOP_CONF_DIR /etc/hadoop/conf
%env HADOOP_YARN_HOME /usr/lib/hadoop
%env HADOOP_MAPRED_HOME /usr/lib/hadoop
%env HADOOP_HDFS_HOME /usr/lib/hadoop-hdfs

%env SPARK_HOME /usr/lib/spark
%env SPARK_CONF_DIR /etc/spark/conf
#%env PYTHONPATH /opt/spark/python/lib/py4j-0.10.7-src.zip
%env PYSPARK_PYTHON /opt/conda/bin/python
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python


env: HDFS_PATH=/user/dominospark/small_data
env: HADOOP_HOME=/usr/lib/hadoop
env: HADOOP_CONF_DIR=/etc/hadoop/conf
env: HADOOP_YARN_HOME=/usr/lib/hadoop
env: HADOOP_MAPRED_HOME=/usr/lib/hadoop
env: HADOOP_HDFS_HOME=/usr/lib/hadoop-hdfs
env: SPARK_HOME=/usr/lib/spark
env: SPARK_CONF_DIR=/etc/spark/conf
env: PYSPARK_PYTHON=/opt/conda/bin/python
env: PYSPARK_DRIVER_PYTHON=/opt/conda/bin/python


In [2]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random
sys.path.insert(1, '/mnt/code/python/')
count=100
hdfs_endpoint='hdfs://10.0.1.76:8020'
dest_folder='/user/dominospark/large-data-temp/'

!hdfs dfs -rmr '/user/dominospark/large-data-temp/'

import generate_data_in_hdfs
args=[str(count),hdfs_endpoint,dest_folder,'local[*]']

generate_data_in_hdfs.main(args)

rmr: DEPRECATED: Please use '-rm -r' instead.
rmr: `/user/dominospark/large-data-temp/': No such file or directory


In [3]:
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write") \
.master('local[*]') \
.config("fs.default.name", "hdfs://10.0.1.76:8020") \
.getOrCreate()


In [4]:
sparkSession

In [5]:

columns = StructType([ StructField("id", IntegerType(), True),
                       StructField("v1", IntegerType(), True),
                       StructField("v2", IntegerType(), True),
                       StructField("v3", IntegerType(), True) ])

df_load = sparkSession.read.csv(dest_folder,schema=columns)
df_load.show()
df_load.count()

+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|  0| 50| 57| 20|
|  1| 57| 52| 49|
|  2|  7| 50| 12|
|  3| 60| 65| 30|
|  4| 66| 31| 85|
|  5| 69| 98| 99|
|  6|  4| 28|  7|
|  7|  7| 11| 83|
|  8| 74| 95| 88|
|  9| 20| 34| 38|
| 10| 11| 20| 86|
| 11| 99| 88| 44|
| 12| 53| 92| 97|
| 13| 74| 79| 76|
| 14| 21| 73| 76|
| 15| 80| 64| 80|
| 16| 69| 86| 33|
| 17| 71|  4| 64|
| 18| 52| 10| 93|
| 19| 30| 65| 28|
+---+---+---+---+
only showing top 20 rows



100

In [6]:
sparkSession.stop()

In [7]:
!hdfs dfs -ls /user/dominospark/large-data-temp/

Found 2 items
-rw-r--r--   1 dominospark dominospark          0 2021-08-13 19:34 /user/dominospark/large-data-temp/_SUCCESS
-rw-r--r--   1 dominospark dominospark       1170 2021-08-13 19:34 /user/dominospark/large-data-temp/part-00000-bfc1dec5-eddd-412d-8e1b-55cb989218fb-c000.csv
